# Этап 5: LoRA (Low-Rank Adaptation) — Хирургическое дообучение

LoRA позволяет адаптировать модель под новую задачу, обучая менее 1% её параметров. 

### Математика:
Мы представляем изменение весов $\Delta W$ как произведение двух матриц низкого ранга:
$$\Delta W = B \cdot A$$
Где $A \in \mathbb{R}^{r \times k}$ и $B \in \mathbb{R}^{d \times r}$, а ранг $r$ очень мал.

**Итоговый выход слоя:**
$$h = W x + \frac{\alpha}{r} (B A) x$$

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from src.model import GPTLanguageModel, device, decode, encode
import copy

class LoRALinear(nn.Module):
    def __init__(self, linear_layer, rank=2, alpha=4):
        super().__init__()
        self.linear = linear_layer
        self.rank = rank
        self.alpha = alpha
        
        # Замораживаем основную матрицу
        self.linear.weight.requires_grad = False
        if self.linear.bias is not None:
            self.linear.bias.requires_grad = False
            
        in_features = self.linear.in_features
        out_features = self.linear.out_features
        target_device = self.linear.weight.device
        
        # Инициализируем A (случайно) и B (нулями)
        # Нулевая инициализация B гарантирует, что в начале обучения 
        # LoRA не влияет на результат (delta = 0)
        self.lora_A = nn.Parameter(torch.randn(in_features, rank, device=target_device) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(rank, out_features, device=target_device))
        self.scaling = alpha / rank

    def forward(self, x):
        return self.linear(x) + ((x @ self.lora_A) @ self.lora_B) * self.scaling

print("✅ LoRA-слой готов к использованию!")

### 2. Подготовка модели (Заморозка + Адаптеры)
Здесь мы сводим количество обучаемых параметров к минимуму.

In [ ]:
def apply_lora(model, rank=2):
    for block in model.blocks:
        for head in block.sa.heads:
            head.query = LoRALinear(head.query, rank=rank)
            head.value = LoRALinear(head.value, rank=rank)
    return model

base_model = GPTLanguageModel().to(device)
base_model.load_state_dict(torch.load('model_ckpt.pt', map_location=device))

# 1. Замораживаем ВСЮ модель
for param in base_model.parameters():
    param.requires_grad = False

# 2. Добавляем адаптеры
lora_model = apply_lora(base_model, rank=2)
lora_model.to(device)

trainable_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in lora_model.parameters())
print(f"Обучаемых параметров: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")

### 3. Обучение (Soft Fine-tuning)
Мы уменьшим количество шагов, чтобы модель не забыла Шекспира окончательно.

In [ ]:
fine_tune_text = "SCENE II. A Laboratory. ROMEO nodes: My circuit is cold, I need more computation!\n" * 30
data_ft = torch.tensor(encode(fine_tune_text), dtype=torch.long, device=device)

# Учим только адаптеры!
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, lora_model.parameters()), lr=1e-3)
lora_model.train()

for i in range(80): # Небольшое количество итераций
    ix = torch.randint(len(data_ft) - 256, (8,))
    x = torch.stack([data_ft[j:j+256] for j in ix])
    y = torch.stack([data_ft[j+1:j+256+1] for j in ix])
    
    logits, loss = lora_model(x, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if i % 20 == 0:
        print(f"Итерация {i}, Loss: {loss.item():.4f}")

### 4. Финальный результат
Теперь модель должна выдавать смесь стилей.

In [ ]:
lora_model.eval()
context = torch.tensor(encode("KING: "), dtype=torch.long, device=device).unsqueeze(0)
print(decode(lora_model.generate(context, max_new_tokens=200)[0].tolist()))